In [51]:
import pandas as pd
from dotenv import load_dotenv
import os
import time
from google.oauth2 import service_account
from googleapiclient.discovery import build
import math

load_dotenv()

gtm_id = os.environ["GTM_ACCOUNT_ID"]

In [52]:
def map_type(type):
    map = {
        '당사 쿠키': 'k',
        '상수': 'c',
        '데이터 영역 변수': 'v',
        'URL': 'u',
        '요소 속성': 'aev',
        '참고표': 'smm',
        'Google 태그: 이벤트 설정': 'gtes',
        '정규식 표': 'remm',
        '요소 속성': 'aev'
    }

    return map[type]
    

# Excel

In [53]:
xlsx_names = ['kyobo', 'library', 'scholar', 'story', 'tree']
service_names = {
    '교보문고_renewal': 'kyobo',
    '전자도서관': 'library',
    '스콜라': 'scholar',
    '스토리': 'story',
    '리딩트리': 'tree'
}

def get_xlsx():
    res = {}
    for name in service_names.values():
        res[name]=pd.ExcelFile(f'./xlsx/{name}.xlsx')

    return res

def get_sheet_name(excel):
    sheet_names = excel.sheet_names
    return sheet_names

def read_sheet(excel_file, sheet_name):
    return pd.read_excel(excel_file, sheet_name=sheet_name)

# GTM

In [54]:
# Get credentials
def get_credentials():
    SERVICE_ACCOUNT_FILE = './xlsx/gtm-api-409502-6f8362f190d7.json'

    credentials_container = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE,
        scopes=["https://www.googleapis.com/auth/tagmanager.edit.containers"]
    )

    return credentials_container

credentials = get_credentials()

# Get service
def get_service():
    service = build('tagmanager', 'v2', credentials = credentials)

    return service

service = get_service()

# Get container
def get_container():
    parent = f"accounts/{gtm_id}"

    containers = service.accounts().containers().list(
        parent=parent,
    ).execute()

    return containers['container']

container_origin = get_container()
containers = {}
for i in container_origin:
    containers[i['name']] = i


# Set Container

In [55]:
service_name = '교보문고_renewal'
container = containers[service_name]

xlsxs = get_xlsx()
xlsx = xlsxs[service_names[service_name]]

# Get workspace
def get_workspaces():
    parent=container['path']

    workspace = service.accounts().containers().workspaces().list(
        parent=parent
    ).execute()

    return workspace['workspace']

workspace_origin = get_workspaces()

workspaces = {}
for i in workspace_origin:
    workspaces[i['name']] = i

workspace = workspaces['new']

In [56]:
def get_variables():
    parent=workspace['path']

    containers = service.accounts().containers().workspaces().variables().list(
        parent=parent
    ).execute()

    return containers

def create_gtm_variable(request_body):
    parent=workspace['path']

    try:
        response = service.accounts().containers().workspaces().variables().create(
            parent=parent,
            body=request_body
        ).execute()

        print(f"Created variable: {response['name']}")
        return response
    except Exception as e:
        print(f"An error occurred: {e}")
        print(f"Above error occurred with: {request_body['name']}")

def list_gtm_variable():
    parent=workspace['path']

    try:
        response = service.accounts().containers().workspaces().variables().list(
            parent=parent
        ).execute()

        print(f"Listed variables: {response['variable']}")
        return response
    except Exception as e:
        print(f"An error occurred: {e}")

In [57]:
list_gtm_variable()

Listed variables: [{'path': 'accounts/6003640738/containers/90556056/workspaces/43/variables/508', 'accountId': '6003640738', 'containerId': '90556056', 'workspaceId': '43', 'variableId': '508', 'name': 'Cookie - ga', 'type': 'k', 'parameter': [{'type': 'template', 'key': 'name', 'value': '_ga'}], 'fingerprint': '1704185130781', 'tagManagerUrl': 'https://tagmanager.google.com/#/container/accounts/6003640738/containers/90556056/workspaces/43/variables/508?apiLink=variable'}, {'path': 'accounts/6003640738/containers/90556056/workspaces/43/variables/509', 'accountId': '6003640738', 'containerId': '90556056', 'workspaceId': '43', 'variableId': '509', 'name': 'Cookie - shtm', 'type': 'k', 'parameter': [{'type': 'template', 'key': 'name', 'value': 'shtm'}], 'fingerprint': '1704185137008', 'tagManagerUrl': 'https://tagmanager.google.com/#/container/accounts/6003640738/containers/90556056/workspaces/43/variables/509?apiLink=variable'}, {'path': 'accounts/6003640738/containers/90556056/workspac

{'variable': [{'path': 'accounts/6003640738/containers/90556056/workspaces/43/variables/508',
   'accountId': '6003640738',
   'containerId': '90556056',
   'workspaceId': '43',
   'variableId': '508',
   'name': 'Cookie - ga',
   'type': 'k',
   'parameter': [{'type': 'template', 'key': 'name', 'value': '_ga'}],
   'fingerprint': '1704185130781',
   'tagManagerUrl': 'https://tagmanager.google.com/#/container/accounts/6003640738/containers/90556056/workspaces/43/variables/508?apiLink=variable'},
  {'path': 'accounts/6003640738/containers/90556056/workspaces/43/variables/509',
   'accountId': '6003640738',
   'containerId': '90556056',
   'workspaceId': '43',
   'variableId': '509',
   'name': 'Cookie - shtm',
   'type': 'k',
   'parameter': [{'type': 'template', 'key': 'name', 'value': 'shtm'}],
   'fingerprint': '1704185137008',
   'tagManagerUrl': 'https://tagmanager.google.com/#/container/accounts/6003640738/containers/90556056/workspaces/43/variables/509?apiLink=variable'},
  {'pat

In [18]:
def create_aev_variable(sheet):
    error = 0
    for row in sheet.iloc():
        if (row[1] == '요소 속성'):
            parameter = [
                {
                    "type": "template",
                    "key": "elementSelector",
                    "value": "ATTRIBUTE"
                },
                {
                    'type': 'template',
                    'key': 'attribute',
                    'value': row[2]
                },
                {
                    'type': 'boolean',
                    'key': 'setDefaultValue',
                    'value': 'false'
                },
            ]

        request_body = {
            "name": row[0],
            "type": 'aev',
            "parameter": parameter
        }

        try:
            create_gtm_variable(request_body)
            time.sleep(5)
        except Exception as e:
            error += 1

    print(f"Total Error: {error}")

In [19]:
def create_dom_variable(sheet):
    error = 0
    for row in sheet.iloc():
        if (row[1] == 'CSS 선택도구'):
            parameter = [
                {
                    "type": "template",
                    "key": "elementSelector",
                    "value":row[2] 
                },
                {
                    'type': 'template',
                    'key': 'selectorType',
                    'value': 'CSS'
                }
            ]

        request_body = {
            "name": row[0],
            "type": 'd',
            "parameter": parameter
        }

        try:
            create_gtm_variable(request_body)
            time.sleep(5)
        except Exception as e:
            error += 1

    print(f"Total Error: {error}")

In [ ]:
# regex
{
   'name': 'TD - 메뉴 - 서비스탭 명칭',
   'type': 'remm',
   'parameter': [
        {
        'type': 'boolean',
        'key': 'setDefaultValue',
        'value': 'false'
        },
        {'type': 'template', 'key': 'input', 'value': '{{Click URL}}'},
        {'type': 'boolean', 'key': 'fullMatch', 'value': 'true'},
        {'type': 'boolean', 'key': 'replaceAfterMatch', 'value': 'true'},
        {'type': 'boolean', 'key': 'ignoreCase', 'value': 'true'},
        {
            'type': 'list',
            'key': 'map',
            'list': 
            [
                {
                'type': 'map',
                'map': 
                [
                    {
                    'type': 'template',
                    'key': 'key',
                    'value': '.*www.kyobobook.co.kr$'
                    },
                    {
                    'type': 'template',
                    'key': 'value',
                    'value': '교보문고'
                    }
                ]
                }
            ]
        }
    ]
}

# table
{
    'name': 'TD - item_list_id',
    'type': 'smm',
    'parameter': [
        {'type': 'boolean',
        'key': 'setDefaultValue',
        'value': 'true'},
        {'type': 'template',
        'key': 'input',
        'value': '{{DLV - ecommerce.item_list_id}}'},
        {'type': 'template',
        'key': 'defaultValue',
        'value': '{{DLV - ecommerce.item_list_id}}'}
    ],
    'formatValue': {
        'convertUndefinedToValue': {
            'type': 'template',
            'value': '{{DLV - item_list_id}}'}
        }
}

In [ ]:
def create_table_variable(sheet):
    error = 0
    for row in sheet.iloc():
        if (math.isnan(row[0]) or row[0] == ' - '):
            map.append({
                'type': 'template',
                'key': 'key',
                'value': row[2]
            })
        else:
            if (len(request_body) == 0):
                try:
                    create_gtm_variable(request_body)
                    time.sleep(5)
                except Exception as e:
                    error += 1

            input = row[2]

            if (row[1] == '참고표'):
                map = []
                type = 'smm'
                
            elif (row[1] == '정규식 표'):
                map = []
                type = 'remm'

            if ()

            request_body = {
                "name": row[0],
                "type": type,
                "parameter": [
                    {
                        'type': 'template',
                        'key': 'input',
                        'value': input
                    },
                ]
            }

        try:
            create_gtm_variable(request_body)
            time.sleep(5)
        except Exception as e:
            error += 1

    print(f"Total Error: {error}")


In [62]:
def create_gtes_variable(sheet):
    error = 0
    request_body = {}
    for row in sheet.iloc():
        if (type(row[0]) is str):
            if (len(request_body) != 0):
                try:
                    create_gtm_variable(request_body)
                    time.sleep(5)
                except Exception as e:
                    error += 1

            request_body['name'] = row[0]
            request_body['type'] = 'gtes'
            eventSettingsTable = []
            userProperties = []

        if (type(row[2]) is str):
            event_data = {
                'type': 'map',
                'map': [
                    {
                        'type': 'template',
                        'key': 'parameter',
                        'value': row[2]
                    },
                    {
                        'type': 'template',
                        'key': 'parameterValue',
                        'value': row[3]
                    }
                ]
            }
            eventSettingsTable.append(event_data)
        if (type(row[4]) is str):
            user_data = {
                'type': 'map',
                'map': [
                    {
                        'type': 'template',
                        'key': 'name',
                        'value': row[4]
                    },
                    {
                        'type': 'template',
                        'key': 'value',
                        'value': row[5]
                    }
                ]
            }
            userProperties.append(user_data)

        request_body['parameter'] = [{
            'type': 'list',
            'key': 'eventSettingsTable',
            'list': eventSettingsTable
        },
        {
            'type': 'list',
            'key': 'userProperties',
            'list': userProperties
        }]

    try:
        create_gtm_variable(request_body)
        time.sleep(5)
    except Exception as e:
        error += 1

    print(f"Total Error: {error}")

In [ ]:
def create_url_variable(sheet):
    error = 0
    for row in sheet.iloc():
        if (row[2] == '검색어'):
            parameter = [
                {
                "type": "template",
                "key": "component",
                "value": "QUERY"
                },
                {
                    'type': 'template',
                    'key': 'queryKey',
                    'value': row[3]
                }
            ]
            type = 'u'

        request_body = {
            "name": row[0],
            "type": type,
            "parameter": parameter
        }

        try:
            create_gtm_variable(request_body)
            time.sleep(5)
        except Exception as e:
            error += 1

    print(f"Total Error: {error}")

In [ ]:
def create_dataLayer_variable(sheet):
    error = 0
    for row in sheet.iloc():
        if (row[1] == '상수'):
            parameter = [{
                'type': 'template',
                'key': 'value',
                'value': row[2],
            }]
            type = 'c'
        elif (row[1] == '맞춤 자바스크립트'):
            parameter = [{
                'type': 'template',
                'key': 'javascript',
                'value': row[2],
            }]
            type = 'jsm'
        elif (row[1] == '당사 쿠키'):
            parameter = [{
                'type': 'template',
                'key': 'name',
                'value': row[2],
            }]
            type = 'k'
        else:
            parameter = [
                {
                    'type': 'template',
                    'key': 'name',
                    'value': row[2],
                },
                {
                    'type': 'integer',
                    'key': 'dataLayerVersion',
                    'value': 2,
                }
                ]
            type = 'v'

        request_body = {
            'name': row[0],
            'type': type,
            'parameter': parameter,
        }

        try:
            create_gtm_variable(request_body)
            time.sleep(5)
        except Exception as e:
            error += 1

    print(f"Error: {error}")


In [21]:
sheet_names = get_sheet_name(xlsx)

for i in sheet_names:
  print(i)

변수(당사쿠키)
변수(데이터 영역)
변수(URL)
변수(자동이벤트 변수)
변수(DOM 요소)
변수(맞춤JS)
변수(참고표)
변수(이벤트 매개변수)
트리거(맞춤 이벤트)
트리거(페이지뷰)
트리거(스크롤 깊이)
트리거(링크 클릭)(Web)
트리거(링크 클릭)(App)
트리거(모든 요소 클릭)(Web)
트리거(모든 요소 클릭)(App)
태그(맞춤HTML)
 태그(Web)
 태그(App 이벤트)
 태그(App 이벤트)


In [63]:
sheet_cookie = read_sheet(xlsx, sheet_names[0])
sheet_data_layer = read_sheet(xlsx, sheet_names[1])
sheet_url = read_sheet(xlsx, sheet_names[2])
sheet_aev = read_sheet(xlsx, sheet_names[3])
sheet_dom = read_sheet(xlsx, sheet_names[4])
sheet_custom_js = read_sheet(xlsx, sheet_names[5])
sheet_table = read_sheet(xlsx, sheet_names[6])
sheet_gtes = read_sheet(xlsx, sheet_names[7])

# create_url_variable(sheet_url)
# create_aev_variable(sheet_aev)
# create_dataLayer_variable(sheet_cookie)
# create_dataLayer_variable(sheet_data_layer)
# create_dataLayer_variable(sheet_custom_js)
# create_dom_variable(sheet_dom)
create_gtes_variable(sheet_gtes)

/var/folders/xq/qf2k1k454qb_jfv62h4qfhn00000gn/T/ipykernel_36771/1799736979.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if (type(row[0]) is str):
/var/folders/xq/qf2k1k454qb_jfv62h4qfhn00000gn/T/ipykernel_36771/1799736979.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  request_body['name'] = row[0]
/var/folders/xq/qf2k1k454qb_jfv62h4qfhn00000gn/T/ipykernel_36771/1799736979.py:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[po

An error occurred: <HttpError 400 when requesting https://tagmanager.googleapis.com/tagmanager/v2/accounts/6003640738/containers/90556056/workspaces/43/variables?alt=json returned "Returned an error response for your request.". Details: "[{'message': 'Returned an error response for your request.', 'domain': 'global', 'reason': 'badRequest'}]">
Above error occurred with: GA4 - Params - 공유
Created variable: GA4 - Params - 상품 목록 공통
Created variable: GA4 - CM - 구매 관련 금권
An error occurred: <HttpError 400 when requesting https://tagmanager.googleapis.com/tagmanager/v2/accounts/6003640738/containers/90556056/workspaces/43/variables?alt=json returned "vendorTemplate.parameter.userProperties: The number of rows must be between 0 and 25.". Details: "[{'message': 'vendorTemplate.parameter.userProperties: The number of rows must be between 0 and 25.\n', 'domain': 'global', 'reason': 'badRequest'}]">
Above error occurred with: GA4 - Params - 로그인
Created variable: GA4 - Params - 마이페이지 주요 버튼 클릭
Creat

In [45]:
for i in sheet_table.iloc(1):
    print(i[3], type(i[3]))

TD - section <class 'str'>
참고표 <class 'str'>
{{DLV - ecommerce.section}} <class 'str'>
nan <class 'float'>
nan <class 'float'>
체크 <class 'str'>
{{DLV - ecommerce.section}} <class 'str'>
undefined를 다음으로 변환... <class 'str'>
{{DLV - section}} <class 'str'>
